In [8]:
import torch
from time import perf_counter
import numpy as np
# 定义模型
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.fc = torch.nn.Linear(64 * 8 * 8, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 64 * 8 * 8)
        x = self.fc(x)
        return x

In [9]:
def timer(f, *args):
    torch.cuda.synchronize()  # 确保之前所有的GPU操作完成
    start = perf_counter()
    f(*args)
    torch.cuda.synchronize()  # 确保当前所有的GPU操作完成
    return 1000 * (perf_counter() - start)


In [80]:
model = MyModel()
times = [timer(model,torch.randn(1, 3, 64, 64)) for _ in range(10000)]
print(np.mean(times))

0.42148077905294484


In [81]:
scripted_model = torch.jit.script(model)
times = [timer(scripted_model,torch.randn(1, 3, 64, 64)) for _ in range(10000)]
print(np.mean(times))

0.3954652471235022


In [55]:
torch.jit.save(scripted_model, '/home/Graph_module/Performance_test/1.pth')

In [56]:
import torch

model = torch.jit.load("/home/Graph_module/Performance_test/1.pth")

In [65]:
model = MyModel()
print(model)


MyModel(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=4096, out_features=10, bias=True)
)


In [70]:
batchsize = 1

In [82]:
scripted_model = torch.jit.script(model)
times = [timer(scripted_model,torch.randn(batchsize, 3, 64, 64)) for _ in range(10000)]
print(np.mean(times))

0.37984403513255527


In [83]:
traced_model = torch.jit.script(model,torch.randn(1, 3, 64, 64))
times = [timer(traced_model,torch.randn(1, 3, 64, 64)) for _ in range(10000)]
print(np.mean(times))

0.38967417483218014


In [84]:
torch.jit.save(traced_model,"/home/Graph_module/Performance_test/traced_model.pt")

In [85]:
torch.jit.save(scripted_model,"/home/Graph_module/Performance_test/scripted_model.pt")

In [1]:
from torch import _dynamo
print(_dynamo.list_backends())

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'openxla_eval', 'tvm']
